In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
7,Covid-19: U.S. Surpasses 10 Million Coronaviru...,https://www.nytimes.com/live/2020/11/08/world/...,"Mon, 09 Nov 2020 04:09:00 GMT",2020-11-09 04:09:00+00:00
1,United States nears 10 million coronavirus cas...,https://www.cnn.com/2020/11/08/health/us-coron...,"Mon, 09 Nov 2020 02:40:00 GMT",2020-11-09 02:40:00+00:00
2,Making a Covid-19 vaccine confidence project j...,https://www.statnews.com/2020/11/08/national-c...,"Mon, 09 Nov 2020 01:07:30 GMT",2020-11-09 01:07:30+00:00
4,Tracking COVID-19 in Alaska: 512 new cases rep...,https://www.adn.com/alaska-news/2020/11/08/tra...,"Sun, 08 Nov 2020 22:01:16 GMT",2020-11-08 22:01:16+00:00
8,Multiple area schools announce closures after ...,https://www.post-gazette.com/news/education/20...,"Sun, 08 Nov 2020 20:42:01 GMT",2020-11-08 20:42:01+00:00
6,Covid Infections in Animals Prompt Scientific ...,https://www.nytimes.com/2020/11/08/science/Cov...,"Sun, 08 Nov 2020 15:42:00 GMT",2020-11-08 15:42:00+00:00
0,COVID-19 Daily Update 11-8-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sun, 08 Nov 2020 15:14:46 GMT",2020-11-08 15:14:46+00:00
9,Latest on COVID-19 in Minnesota: State ends we...,https://www.mprnews.org/story/2020/11/08/lates...,"Sun, 08 Nov 2020 11:00:00 GMT",2020-11-08 11:00:00+00:00
5,"With Covid-19 cases surging, the U.S. is in fo...",https://www.nbcnews.com/health/health-news/cov...,"Sun, 08 Nov 2020 09:30:00 GMT",2020-11-08 09:30:00+00:00
3,COVID-19 Data Review Panel 11-7-2020 West Vir...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sun, 08 Nov 2020 04:27:48 GMT",2020-11-08 04:27:48+00:00
